In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [2]:

metadata_df_path_whiteCAT = "/workspace/project/catatonia_VAE-main_bq/metadata_20250110/meta_data_whiteCAT_all_variables.csv"
metadata_df_path_NSS = "/workspace/project/catatonia_VAE-main_bq/metadata_20250110/meta_data_NSS_all_variables.csv"
metadata_df_wC = pd.read_csv(metadata_df_path_whiteCAT)
metadata_df_NSS = pd.read_csv(metadata_df_path_NSS)

In [3]:

def extract_dois_by_diagnosis_df_wC(df):
    """Extrahiert DOIs gruppiert nach Diagnose als DataFrame"""
    return df.groupby('Diagnosis')['duration_illness'].apply(lambda x: x.tolist()).reset_index()

def extract_dois_by_diagnosis_df_NSS(df):
    """Extrahiert DOIs gruppiert nach Diagnose als DataFrame"""
    return df.groupby('Diagnosis')['DOI'].apply(lambda x: x.tolist()).reset_index()


def analyze_disease_duration(df, dataset_name):
    """
    Analysiert die Krankheitsdauer nach Diagnosen
    """
    print(f"\n{'='*50}")
    print(f"ANALYSE: {dataset_name}")
    print(f"{'='*50}")
    
    # Verfügbare Spalten anzeigen (um die richtige Krankheitsdauer-Spalte zu finden)
    print("Verfügbare Spalten:")
    print(df.columns.tolist())
    
    # Häufige Spaltennamen für Krankheitsdauer
    duration_columns = [col for col in df.columns if any(keyword in col.lower() 
                       for keyword in ['DOI', 'duration_illness'])]
    
    print(f"\nMögliche Krankheitsdauer-Spalten: {duration_columns}")
    
    return duration_columns

def create_duration_analysis(df, duration_col, diagnosis_col='Diagnose', dataset_name='Dataset'):
    """
    Erstellt umfassende Analyse der Krankheitsdauer
    """
    # Daten bereinigen
    clean_df = df.dropna(subset=[duration_col, diagnosis_col])
    
    print(f"\n=== STATISTISCHE ÜBERSICHT - {dataset_name} ===")
    
    # Grundlegende Statistiken
    duration_stats = clean_df.groupby(diagnosis_col)[duration_col].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(2)
    
    print(duration_stats)
    return duration_stats, clean_df



def compare_datasets(df_wC, df_NSS, duration_col_wC, duration_col_NSS):
    """
    Vergleicht die Krankheitsdauer zwischen WhiteCAT und NSS
    """
    print(f"\n{'='*60}")
    print("VERGLEICH ZWISCHEN WHITECAT UND NSS")
    print(f"{'='*60}")
    
    # Gesamtvergleich
    wC_durations = df_wC[duration_col_wC].dropna()
    NSS_durations = df_NSS[duration_col_NSS].dropna()
    
    print(f"WhiteCAT - Durchschnittliche Krankheitsdauer: {wC_durations.mean():.2f}")
    print(f"NSS - Durchschnittliche Krankheitsdauer: {NSS_durations.mean():.2f}")
    
    # Statistischer Test
    t_stat, p_value = stats.ttest_ind(wC_durations, NSS_durations)
    print(f"\nt-Test zwischen Datasets:")
    print(f"t-Statistik: {t_stat:.4f}")
    print(f"p-Wert: {p_value:.4f}")
    
    if p_value < 0.05:
        if wC_durations.mean() > NSS_durations.mean():
            print("→ WhiteCAT-Patienten sind signifikant länger erkrankt")
        else:
            print("→ NSS-Patienten sind signifikant länger erkrankt")
    else:
        print("→ Kein signifikanter Unterschied zwischen den Datasets")

In [4]:
dois_df_wC = extract_dois_by_diagnosis_df_wC(metadata_df_wC)
dois_df_NSS = extract_dois_by_diagnosis_df_NSS(metadata_df_NSS)

In [5]:
duration_col_wC = 'duration_illness'
duration_col_NSS = 'DOI'
stats_wC, clean_wC = create_duration_analysis(metadata_df_wC, duration_col_wC, 'Diagnosis', 'WhiteCAT')
stats_NSS, clean_NSS = create_duration_analysis(metadata_df_NSS, duration_col_NSS, 'Diagnosis', 'NSS')


=== STATISTISCHE ÜBERSICHT - WhiteCAT ===
           count   mean  median    std  min   max
Diagnosis                                        
CTT           63  14.75    10.0  13.55  0.0  49.0
SCHZ          95  11.89    11.0  10.20  0.0  46.0

=== STATISTISCHE ÜBERSICHT - NSS ===
           count   mean  median    std  min   max
Diagnosis                                        
CTT           12  10.33     6.5  11.49  0.0  37.0
SCHZ         105  10.70     8.0  10.81  0.0  40.0


In [6]:

compare_datasets(clean_wC, clean_NSS, duration_col_wC, duration_col_NSS)


VERGLEICH ZWISCHEN WHITECAT UND NSS
WhiteCAT - Durchschnittliche Krankheitsdauer: 13.03
NSS - Durchschnittliche Krankheitsdauer: 10.66

t-Test zwischen Datasets:
t-Statistik: 1.7165
p-Wert: 0.0872
→ Kein signifikanter Unterschied zwischen den Datasets
